In [ ]:
from azure.storage.blob import BlobServiceClient
import os
import pymssql
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
from sqlalchemy import true
import requests as req
import pyodbc
import sqlalchemy as sal
from urllib.parse import quote_plus
from datetime import timedelta, date, datetime
import re
from urllib.parse import quote_plus
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine, MetaData
from urllib.parse import quote_plus
import time

# Parâmetros de conexão do blob storage
STORAGEACCOUNTURL = "berstgpbi.blob.core.windows.net"
STORAGEACCOUNTKEY = "sp=racwdl&st=2023-10-03T16:53:11Z&se=2030-03-11T00:53:11Z&spr=https&sv=2022-11-02&sr=c&sig=S1kMNLavP2Uwz%2BqB6z1HqurSn5ZW7XpCjki3IW%2BjX8A%3D"
CONTAINERNAME = "foccoevviva"

# String de conexão Windows Server.
parametros_db = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=BERMATBD02\MSSQLDW,1453;'
    'DATABASE=DW_BERTOLINI_DEV;'
    'Trusted_Connection=yes;'
)

# Convertendo a string para um padrão de URI HTML.
url_db = quote_plus(parametros_db)
# Conexão.
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % url_db)

# Objeto de metadados
metadata = MetaData()
metadata.reflect(bind=engine)

# Criar instância do cliente BlobService
blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# Obter cliente do contêiner
container_client = blob_service_client_instance.get_container_client(container=CONTAINERNAME)

# Listar blobs no contêiner
blob_list = container_client.list_blobs()

# Lista para armazenar DataFrames
tabelas = []

# Iterar sobre blobs no contêiner
for blob in blob_list:
    try:
        # Verificar se o blob começa com 'PROJETO_AMB_ITENS'
        if 'PROJETO_AMB_ITENS' in blob.name:
            # Obter um objeto de leitura do blob
            blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, blob.name)
            download_stream = blob_client_instance.download_blob()

            # Lê o conteúdo do blob diretamente usando iso-8859-1 (latin-1) como codificação
            conteudo = download_stream.readall().decode('iso-8859-1')
            print('conteúdo lido')
            # Cria um dataframe a partir do conteúdo da tabela
            df = pd.read_csv(StringIO(conteudo), sep=';', on_bad_lines="skip")

            # Adiciona a coluna 'ano_mes' ao dataframe
            df['ano_mes'] = blob.name[-11:-4]
            print('coluna ano_mes adicionada')
            # Armazena o dataframe na lista de tabelas
            tabelas.append(df)

    except Exception as e:
        print(f'Ocorreu um erro ao processar o blob {blob.name}: {e}')

# Verifica se há DataFrames disponíveis antes de tentar a concatenação
if tabelas:
    # Faz o merge das tabelas usando a coluna 'ano_mes'
    df_consolidado = pd.concat(tabelas, ignore_index=True)
    print('merge OK')
    # Grava a tabela consolidada diretamente no banco de dados
    df_consolidado.to_sql(name='PROJETO_AMBIENTES_CONSOLIDADA', schema='sfl', con=engine, index=False, if_exists='replace')

    print('Dados consolidados foram salvos no banco de dados.')
    time.sleep(3)
else:
    print('Nenhuma tabela encontrada para consolidar.')
time.sleep(3)